In [1]:
import cv2
from rtmlib import Wholebody, draw_skeleton
import random
import pandas as pd
import os
from tqdm import tqdm
import wget
import json

In [2]:
device = 'cpu'  # cpu, cuda, mps
backend = 'onnxruntime'  # opencv, onnxruntime, openvino
img = cv2.imread('../imgs/000000004359.jpg')

In [7]:
anns_df = pd.read_csv('data/coco_diff/single_anns/person_anns.csv')
img_url_list = list(anns_df['img_url'].unique())
random.seed(24) # to use same collection of images
img_sample = random.sample(img_url_list, 2000)

In [ ]:
old_cwd = os.getcwd()
new_cwd = old_cwd+'/imgs'
os.chdir(new_cwd)
for url in tqdm(img_sample):
    try:
        file_name = url.split('/')[-1]
        file_path = os.path.join(new_cwd, file_name)
        
        if not os.path.exists(file_path):
            wget.download(url)
    except Exception as e:
        print(f"\nError in download of {url}: {e}")
os.chdir(old_cwd)

In [68]:
img_source = './imgs'

In [69]:
openpose_skeleton = True 

wholebody = Wholebody(to_openpose=openpose_skeleton,
                      mode='balanced', 
                      backend=backend, device=device)

keypoints_list = []
for filename in tqdm(os.listdir(img_source)):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(img_source, filename)  
        image = cv2.imread(image_path)
        
        if image is None:
            print(f"Error in image loading: {filename}")
            continue

    keypoints, scores = wholebody(image)
    img_id = filename.split('\\')[-1].strip('.jpg').lstrip('0')
    frame_obj = {'Img_ID': img_id, 'people': []}
    for index, (person_kp, person_score) in enumerate(zip(keypoints, scores)):
                    person_keypoints = [{"x": float(kp[0]), "y": float(kp[1]), "score": float(sc)} for kp, sc in zip(person_kp, person_score)]
                    frame_obj[f"people"].append(person_keypoints)
    keypoints_list.append(frame_obj)

load C:\Users\fedep\.cache\rtmlib\hub\checkpoints\yolox_m_8xb8-300e_humanart-c2c7a14a.onnx with onnxruntime backend
load C:\Users\fedep\.cache\rtmlib\hub\checkpoints\rtmw-x_simcc-cocktail13_pt-ucoco_270e-256x192-fbef0d61_20230925.onnx with onnxruntime backend


100%|██████████| 2001/2001 [1:11:30<00:00,  2.14s/it] 


In [70]:
with open("HPE/COCO/keypoints.json", "w") as f:
    json.dump(keypoints_list, f, indent=4)